In [24]:
import random
import numpy as np

def knapsack_genetic(v, w, K, retain_count=1, N=50, s=0.5, c=0.8, m=0.1, maxIter=15):
    num_items = len(v)
    
    random.seed(42)  # Semilla para reproducibilidad
    
    # Generar la población inicial de individuos aleatorios
    def generate_individual():
        return [random.randint(0, 1) for _ in range(num_items)]
    
    def generate_population(size):
        return [generate_individual() for _ in range(size)]

    # Función de fitness que evalúa el valor total y penaliza el exceso de peso
    def fitness(individual):
        total_value = sum(v[i] * individual[i] for i in range(num_items))
        total_weight = sum(w[i] * individual[i] for i in range(num_items))
        if total_weight > K:
            return 0  # Penalización
        return total_value
    
    def sort_population(population):
        graded = [(fitness(individual), individual) for individual in population]
        graded = [x[1] for x in sorted(graded, key=lambda x: x[0], reverse=True)]
        return graded

    # Selección oara mutar y cruzar por distribución de probabilidad
    def selection(sorted_population):
        # Generar probabilidades con un decaimiento logarítmico
        prob = np.logspace(0, -1, len(sorted_population))
        prob /= prob.sum() # normalizar
        
        survivals_num = int(len(sorted_population) * s)
        
        flat_population = np.array(sorted_population)
            
        # Seleccionar sobrevivientes
        survivals = flat_population[np.random.choice(flat_population.shape[0], size=survivals_num, p=prob, replace=False)]
        
        return survivals.tolist()

    # Cruce de un punto
    def crossover(parent1, parent2):
        point = random.randint(1, num_items - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

    # Mutación con probabilidad m
    def mutate(individual, mutation_rate):
        for i in range(num_items):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]  # Cambia 0 a 1 o 1 a 0
        return individual

    # Algoritmo principal
    population = generate_population(N)
    best_solution = None
    best_value = 0
    
    for _ in range(maxIter):
        # Evaluar la población actual
        population = sort_population(population)
        if fitness(population[0]) > best_value:
            best_solution = population[0]
            best_value = fitness(population[0])

        # Selección
        selected = selection(population)

        # Conservar a los individuos más aptos
        population = population[:retain_count]
        
        # Cruce para generar nuevos individuos
        children = []
        while len(children) < int(N * c):
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            if parent1 != parent2:
                child1, child2 = crossover(parent1, parent2)
                children.append(child1)
                children.append(child2)

        # Mutación
        mutated = []
        for i in range(int(N * m)):
            ind = random.choice(selected)
            mutated.append(mutate(ind, m))

        # Crear la nueva población
        population = population + children + mutated

    # Resultados
    selected_items = [i for i in range(num_items) if best_solution[i] == 1]
    total_value = sum(v[i] for i in selected_items)
    total_weight = sum(w[i] for i in selected_items)
    
    return selected_items, total_value, total_weight


In [33]:
# Valores y pesos de los items
valores = [10, 12, 8, 5, 8, 5, 6, 7, 6, 12, 8, 8, 10, 9, 8, 3, 7, 8, 5, 6]
pesos = [6, 7, 7, 3, 5, 2, 4, 5, 3, 9, 8, 7, 8, 6, 5, 2, 3, 5, 4, 6]
capacidad = 50

# Parámetros
N=100 # Tamaño de la población
retain_count=int(0.01 * N) # Número de individuos a mantener
s=0.2 # Porcentaje de supervivencia
c=0.8 # Porcentaje de cruce
m=0.1 # Porcentaje de mutación
maxIter=1000 # Número máximo de iteraciones

# Llamada al algoritmo genético
items_seleccionados, valor_total, peso_total = knapsack_genetic(valores, pesos, capacidad, retain_count, N, s, c, m, maxIter)

print("Valor total:", valor_total)
print("Peso total:", peso_total)
print("\nItems seleccionados:")
for i in items_seleccionados:
    print(f"Ítem {i+1}: valor={valores[i]}, peso={pesos[i]}")

Ítems seleccionados: [0, 1, 3, 4, 5, 7, 8, 13, 14, 16, 17]
Valor total: 85
Peso total: 50
Ítem 1: valor=10, peso=6
Ítem 2: valor=12, peso=7
Ítem 4: valor=5, peso=3
Ítem 5: valor=8, peso=5
Ítem 6: valor=5, peso=2
Ítem 8: valor=7, peso=5
Ítem 9: valor=6, peso=3
Ítem 14: valor=9, peso=6
Ítem 15: valor=8, peso=5
Ítem 17: valor=7, peso=3
Ítem 18: valor=8, peso=5
